<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 09:37:06
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 8.90 K (0.07%)
Current PnL: -13.26 L (-9.24%)
CY Booked + Current PnL: -6.52 L (-4.54%)
-------------------
Total profit:  4.71 L
Total loss:  -17.97 L
-------------------
Total Booked + Current PnL: 20.46 L (17.55%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.67 L (52.45%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,93.75,133002.0,-9477.0,10095.0,0.29,-6.65,7.59,0.44,243.0,-0.94,0.97,56.98,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,32.29,101814.0,23452.0,10578.0,0.20,29.93,10.39,43.43,77.0,2.22,0.75,36.84,XY24,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,82.29,110672.0,6360.0,11178.0,-0.17,6.10,10.10,16.81,27.0,0.57,0.81,23.12,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-17.07,H-LC,86.46,112995.0,9429.0,11277.0,-0.42,9.10,9.98,20.00,30.0,0.84,0.83,24.46,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173893.0,33053.0,11442.0,0.89,23.47,6.58,31.59,89.0,2.89,1.27,51.05,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.63,L-SC,98.96,141918.0,76.0,56498.0,0.37,0.05,39.81,39.88,259.0,0.0,1.04,43.82,XY24,NTT,POWER
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,54.17,151018.0,105.0,60347.0,0.10,0.07,39.96,40.06,102.0,0.0,1.11,23.52,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,9.38,49076.0,7821.0,2984.0,4.95,18.96,6.08,26.19,1.0,2.62,0.36,15.54,XY24,ATH,IT
16,BLUSPRING,226.45,84.55,NaN,NaN,18.75,18083.0,-31510.0,434.0,4.81,-63.54,2.40,-62.66,NaN,-72.60,0.13,21.27,XY24,ATH,MISC
78,SURYODAY,189.78,240.00,57.89,H-SC,15.62,112322.0,-37604.0,77278.0,2.63,-25.08,68.80,26.46,145.0,-0.49,0.82,43.89,XR,NTT,BANKS
70,SATIN,199.77,274.00,-14.59,H-SC,88.54,167555.0,-28419.0,101237.0,2.60,-14.50,60.42,37.16,130.0,-0.28,1.23,25.13,XY24,NTT,FINANCE
56,LAOPALA,369.40,464.00,129.14,H-SC,85.42,72700.0,-28146.0,53972.0,2.01,-27.91,74.24,25.61,136.0,-0.52,0.53,31.12,AR,NTT,CERAMICS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-167.05,M-SC,59.38,76746.0,-25464.0,118857.0,-5.76,-24.91,154.87,91.37,206.0,-0.21,0.56,24.58,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-29.71,H-SC,73.96,210099.0,-38431.0,253905.0,-1.53,-15.46,120.85,86.70,129.0,-0.15,1.54,38.99,XY24,NTT,BANKS
90,VALIANTORG,512.64,838.00,-350.78,H-SC,4.17,106704.0,-26582.0,111175.0,-1.39,-19.94,104.19,63.47,148.0,-0.24,0.78,78.86,XR,NTT,CHEMICALS
84,TITAGARH,1097.23,1548.00,0.31,H-SC,39.06,164360.0,-27655.0,106538.0,-1.19,-14.40,64.82,41.08,152.0,-0.26,1.20,39.11,XY24,NTT,ENGINEERING
74,SIEMENS,4406.90,7969.85,-3.07,X-LC,25.00,81875.0,-28297.0,117368.0,-1.11,-25.68,143.35,80.85,14.0,-0.24,0.60,20.59,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,93.75,133002.0,-9477.0,10095.0,0.29,-6.65,7.59,0.44,243.0,-0.94,0.97,56.98,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.51,M-SC,10.42,148082.0,-23059.0,22997.0,0.11,-13.47,15.53,-0.03,189.0,-1.00,1.08,6.43,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,89.58,148800.0,-25759.0,25787.0,-0.11,-14.76,17.33,0.02,127.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-211.87,H-SC,95.83,80662.0,-14654.0,14697.0,0.15,-15.37,18.22,0.04,154.0,-1.00,0.59,59.10,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.65,X-MC,12.50,86450.0,-16202.0,16201.0,-0.64,-15.78,18.74,-0.00,58.0,-1.00,0.63,31.41,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,9.38,49076.0,7821.0,2984.0,4.95,18.96,6.08,26.19,1.0,2.62,0.36,15.54,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.70,X-LC,34.38,264748.0,-31186.0,125093.0,-0.19,-10.54,47.25,31.73,1.0,-0.25,1.94,5.97,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.91,X-LC,60.42,289139.0,27538.0,118084.0,-0.21,10.53,40.84,55.67,2.0,0.23,2.12,17.27,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.03,X-LC,27.08,260120.0,-10227.0,44845.0,0.39,-3.78,17.24,12.81,6.0,-0.23,1.90,8.79,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.47,X-LC,29.17,277582.0,12156.0,44746.0,-0.25,4.58,16.12,21.44,9.0,0.27,2.03,11.53,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,30.21,173893.0,33053.0,11442.0,0.89,23.47,6.58,31.59,89.0,2.89,1.27,51.05,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.31,M-LC,32.29,101814.0,23452.0,10578.0,0.20,29.93,10.39,43.43,77.0,2.22,0.75,36.84,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,77.08,73630.0,39181.0,766.0,0.78,113.74,1.04,115.97,NaN,51.15,0.54,12.59,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,52.08,232987.0,8025.0,159619.0,-0.88,3.57,68.51,74.53,185.0,0.05,1.71,35.27,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.99,25.90,H-SC,41.15,128961.0,3664.0,55505.0,-0.29,2.92,43.04,47.22,139.0,0.07,0.94,61.34,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.87,H-SC,47.92,119000.0,16996.0,28024.0,-0.20,16.66,23.55,44.14,124.0,0.61,0.87,35.61,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,32.29,101814.0,23452.0,10578.0,0.20,29.93,10.39,43.43,77.0,2.22,0.75,36.84,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.21,173893.0,33053.0,11442.0,0.89,23.47,6.58,31.59,89.0,2.89,1.27,51.05,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,77.08,73630.0,39181.0,766.0,0.78,113.74,1.04,115.97,NaN,51.15,0.54,12.59,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,90.62,205535.0,33914.0,128151.0,-0.25,19.76,62.35,94.43,198.0,0.26,1.50,51.70,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,81.25,166563.0,15618.0,93842.0,0.60,10.35,56.34,72.51,69.0,0.17,1.22,14.73,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.89,H-SC,80.21,135516.0,12180.0,121585.0,-0.90,9.88,89.72,108.45,122.0,0.10,0.99,34.44,AR,ATH,MISC
54,KPIGREEN,497.21,731.99,25.90,H-SC,41.15,128961.0,3664.0,55505.0,-0.29,2.92,43.04,47.22,139.0,0.07,0.94,61.34,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,18.75,18083.0,-31510.0,434.0,4.81,-63.54,2.40,-62.66,NaN,-72.60,0.13,21.27,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67834.0,-45715.0,85769.0,0.71,-40.26,126.44,35.27,261.0,-0.53,0.50,66.02,XR,NTT,HOTELS
58,MASFIN,326.60,399.50,-18.70,H-SC,75.00,92460.0,-5520.0,27387.0,0.26,-5.63,29.62,22.32,137.0,-0.20,0.68,34.10,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,7.60,X-SC,100.00,100216.0,-747.0,95275.0,-0.45,-0.74,95.07,93.63,64.0,-0.01,0.73,49.27,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.31,M-LC,32.29,101814.0,23452.0,10578.0,0.20,29.93,10.39,43.43,77.0,2.22,0.75,36.84,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,7.60,X-SC,100.00,100216.0,-747.0,95275.0,-0.45,-0.74,95.07,93.63,64.0,-0.01,0.73,49.27,SR,ATH,CHEMICALS
52,JPPOWER,18.73,26.20,-29.63,L-SC,98.96,141918.0,76.0,56498.0,0.37,0.05,39.81,39.88,259.0,0.00,1.04,43.82,XY24,NTT,POWER
15,BLUESTARCO,1646.70,2337.55,-1.13,H-SC,97.92,185440.0,20770.0,48307.0,0.66,12.61,26.05,41.95,121.0,0.43,1.36,21.02,X40N,ATH,AC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,67834.0,-45715.0,85769.0,0.71,-40.26,126.44,35.27,261.0,-0.53,0.50,66.02,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-211.87,H-SC,95.83,80662.0,-14654.0,14697.0,0.15,-15.37,18.22,0.04,154.0,-1.00,0.59,59.10,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.86
1,25,41.90
2,50,70.45


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.42
LC    33.90
MC    21.64
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.52
X40      12.23
XY25     11.97
X40N     11.94
XR       10.12
AR        8.22
OX40N     8.22
MH        1.69
X5K       1.46
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.64
M-SC    15.93
H-LC    14.11
X-LC    12.11
H-MC     9.57
X-MC     7.61
M-LC     6.63
X-SC     4.40
M-MC     4.10
L-SC     2.45
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-1.82,35.88
IT,12.09,-9.97,63.97
FINANCE,7.89,-12.04,52.51
BANKS,7.86,-6.11,49.41
MISC,6.61,-14.88,68.07
PAINTS,5.73,-11.68,33.69
ELECTRICAL,5.53,2.34,43.27
HEALTHCARE,4.56,-3.65,33.92
AUTO,4.33,-14.37,64.66


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2698216.0,27
XR,1075541.0,13
AR,1021402.0,10
X40,650236.0,9
X40N,532980.0,11
XY25,484487.0,8
OX40N,306174.0,11
SR,176864.0,2
MH,84750.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1964150.0,20
M-SC,1761353.0,21
H-LC,612100.0,12
H-MC,529703.0,8
X-LC,521509.0,7
X-MC,413240.0,6
M-MC,411237.0,3
X-SC,317815.0,5
M-LC,267809.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      869390.0      6
M-SC       XY24      814976.0      7
H-SC       AR        520609.0      3
M-SC       XR        331853.0      4
M-MC       XY24      301754.0      2
H-MC       XY24      297104.0      3
X-LC       X40       243177.0      2
H-SC       XR        215840.0      3
           X40N      213102.0      4
H-LC       X40N      196047.0      4
M-SC       AR        182321.0      2
           OX40N     160224.0      5
M-LC       XY24      158825.0      3
H-LC       X40       156353.0      3
X-MC       X40       147592.0      2
L-SC       XR        147407.0      2
X-SC       XR        123477.0      1
X-LC       XY25      118880.0      3
M-SC       XY25      118857.0      1
X-LC       AR        117368.0      1
M-MC       XR        109483.0      1
X-MC       XY25      103110.0      1
X-SC       SR         95275.0      1
X-MC       XY24       94192.0      1
M-LC       XR         93842.0      1
H-SC       OX40N      89704.0      3
H-MC       XY25       81824.0      1
M-SC       SR         81589.0      1
H-LC       AR         80101.0      1
M-SC       X40        71533.0      1
H-LC       X200       68590.0      1
           X5K        67968.0      2
H-MC       AR         60347.0      1
L-SC       AR         59890.0      1
X-SC       XY24       59018.0      1
L-SC       XY24       56498.0      1
H-SC       MH         55505.0      1
L-MC       XR         53639.0      1
X-MC       X40N       52145.0      1
L-LC       XY25       46674.0      1
H-LC       XY24       43041.0      1
X-LC       X40N       42084.0      1
X-SC       OX40N      40045.0      2
H-MC       X40        31581.0      1
           X40N       29602.0      1
           MH         29245.0      1
X-MC       OX40N      16201.0      1
M-LC       XY25       15142.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
